## 13-9. 프로젝트 - Movielens 영화 추천 실습

이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보겠습니다.

이번에 활용할 데이터셋은 추천 시스템의 MNIST라고 부를만한 Movielens 데이터입니다.

- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다.
- 별점을 시청횟수로 해석해서 생각하겠습니다.
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

Cloud Storage에 미리 업로드된 ml-1m폴더 내 파일을 심볼릭 링크로 개인 storage에 연결해 줍니다.

In [1]:
# ! mkdir -p ~/aiffel/recommendata_iu/data/ml-1m
# ! ln -s ~/data/ml-1m/* ~/aiffel/recommendata_iu/data/ml-1m

In [123]:
import numpy as np
import scipy
import implicit
import pandas as pd
import datetime as dt

print(np.__version__)
print(scipy.__version__)
print(implicit.__version__)

1.21.4
1.7.1
0.4.8


### 1) 데이터 준비와 전처리

----------------------------

Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [206]:
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [207]:
ratings.drop(columns = "timestamp", inplace = True)

In [208]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [209]:
# ratings 데이터 확인
ratings.head()

,user_id,movie_id,ratings
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [210]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [211]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


여기까지가 전처리입니다. 이후에는 이전 스텝에 소개했던 것과 동일한 방식으로 MF model을 구성하여 내가 좋아할 만한 영화를 추천해 볼 수 있습니다.

In [212]:
df = pd.merge(ratings, movies)  # 그냥 merge를 해줘도 
df

,user_id,movie_id,counts,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...
836473,5851,3607,5,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026,4,Slaughterhouse (1987),Horror
836475,5854,690,3,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909,4,"Five Wives, Three Secretaries and Me (1998)",Documentary


In [213]:
df_movies = df[["user_id", "title", "counts"]]
df_movies

,user_id,title,counts
0,1,One Flew Over the Cuckoo's Nest (1975),5
1,2,One Flew Over the Cuckoo's Nest (1975),5
2,12,One Flew Over the Cuckoo's Nest (1975),4
3,15,One Flew Over the Cuckoo's Nest (1975),4
4,17,One Flew Over the Cuckoo's Nest (1975),5
...,...,...,...
836473,5851,One Little Indian (1973),5
836474,5854,Slaughterhouse (1987),4
836475,5854,"Promise, The (Versprechen, Das) (1994)",3
836476,5938,"Five Wives, Three Secretaries and Me (1998)",4


### 2) 분석해 봅시다.

-----------------------------

- df에 있는 유니크한 영화 개수
- df에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

In [214]:
# 유니크한 영화개수 유니크한 사용자 수
print("df 데이터에 있는 유니크한 영화개수 : ",df["movie_id"].nunique())
print("df 데이터에 있는 유니크한 영화개수 : ", df["user_id"].nunique())

df 데이터에 있는 유니크한 영화개수 :  3628
df 데이터에 있는 유니크한 영화개수 :  6039


In [215]:
temp = df.groupby(["movie_id"])[["counts"]].sum()
a = temp.sort_values(ascending = False, by = "counts").head(30)

# top 30 영화
b = [i for i in a.index]
movies.iloc[b]

,movie_id,title,genre
2858,2927,Brief Encounter (1946),Drama|Romance
260,263,Ladybird Ladybird (1994),Drama
1196,1214,Alien (1979),Action|Horror|Sci-Fi|Thriller
2028,2097,Something Wicked This Way Comes (1983),Children's|Horror
1210,1228,Raging Bull (1980),Drama
1198,1216,"Big Blue, The (Le Grand Bleu) (1988)",Adventure|Romance
593,597,Pretty Woman (1990),Comedy|Romance
2571,2640,Superman (1978),Action|Adventure|Sci-Fi
2762,2831,"Dog of Flanders, A (1999)",Drama
589,593,"Silence of the Lambs, The (1991)",Drama|Thriller


### 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.

In [216]:
# 영화 찾기
movies[movies["title"].str.lower().str.contains("toy")]

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1948,2017,Babes in Toyland (1961),Children's|Fantasy|Musical
2184,2253,Toys (1992),Action|Comedy|Fantasy
2411,2480,Dry Cleaning (Nettoyage à sec) (1997),Drama
3017,3086,March of the Wooden Soldiers (a.k.a. Laurel & ...,Comedy
3045,3114,Toy Story 2 (1999),Animation|Children's|Comedy


In [217]:
# 내가 좋아하는 영화
my_favorite = ["Toy Story (1995)", "Jumanji (1995)", "Amadeus (1984)", "When Harry Met Sally... (1989)", "Lion King, The (1994)"]

my_movies = pd.DataFrame({"user_id" : ["Taejong"]*5, "title": my_favorite, "counts" : [5, 4, 4, 5, 4]})

if not df_movies.isin({"user_id" : ["Taejong"]})["user_id"].any():
    df_movies = df_movies.append(my_movies)

df_movies.tail(10)

,user_id,title,counts
836473,5851,One Little Indian (1973),5
836474,5854,Slaughterhouse (1987),4
836475,5854,"Promise, The (Versprechen, Das) (1994)",3
836476,5938,"Five Wives, Three Secretaries and Me (1998)",4
836477,5948,Identification of a Woman (Identificazione di ...,5
0,Taejong,Toy Story (1995),5
1,Taejong,Jumanji (1995),4
2,Taejong,Amadeus (1984),4
3,Taejong,When Harry Met Sally... (1989),5
4,Taejong,"Lion King, The (1994)",4


In [218]:
df_movies

,user_id,title,counts
0,1,One Flew Over the Cuckoo's Nest (1975),5
1,2,One Flew Over the Cuckoo's Nest (1975),5
2,12,One Flew Over the Cuckoo's Nest (1975),4
3,15,One Flew Over the Cuckoo's Nest (1975),4
4,17,One Flew Over the Cuckoo's Nest (1975),5
...,...,...,...
0,Taejong,Toy Story (1995),5
1,Taejong,Jumanji (1995),4
2,Taejong,Amadeus (1984),4
3,Taejong,When Harry Met Sally... (1989),5


In [219]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique= df_movies["user_id"].unique()
movies_unique = df_movies["title"].unique()

# 유저, 영화 indexing 하는 코드 
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movies_unique)}

In [234]:
len(df_movies["user_id"].unique())

6040

In [220]:
movie_to_idx["When Harry Met Sally... (1989)"]  # 잘 작동한다.

488

In [221]:
user_to_idx["Taejong"]

6039

In [236]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = df_movies['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(df_movies):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    df_movies['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = df_movies['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(df_movies):
    print('movie column indexing OK!!')
    df_movies['title'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

df_movies

user_id column indexing OK!!
movie column indexing OK!!


,user_id,title,counts
0,0,0,5
1,1,0,5
2,2,0,4
3,3,0,4
4,4,0,5
...,...,...,...
0,6039,40,5
1,6039,513,4
2,6039,100,4
3,6039,488,5


In [19]:
# 내가 만든 유저 인덱싱화 함수
# def usertoidx(x):
#     idx = user_to_idx[x]
#     return idx

# ratings["user_id"] = ratings["user_id"].apply(usertoidx)

### 4) CSR matrix를 직접 만들어 봅시다.

In [242]:
print("df_movies.user_id.shape : ",df_movies.user_id.shape)
print("df_movies.movie_id.shape : ", df_movies.title.shape)

df_movies.user_id.shape :  (836483,)
df_movies.movie_id.shape :  (836483,)


In [241]:
df_movies[df_movies["title"] == 1]

,user_id,title,counts
1680,0,1,3
1681,15,1,3
1682,16,1,5
1683,1680,1,4
1684,18,1,5
...,...,...,...
2118,1659,1,3
2119,1938,1,5
2120,1939,1,5
2121,1672,1,4


In [244]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = df_movies['user_id'].nunique()
num_movie = df_movies['title'].nunique()

csr_data = csr_matrix((df_movies.counts, (df_movies.user_id, df_movies.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

### 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

- 참고자료1 : [document](https://implicit.readthedocs.io/en/latest/als.html)
- 참고자료2 : [모델 설명](https://yeomko.tistory.com/4)  

In [245]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [277]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=256, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [278]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [279]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

### 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [280]:
# 나(Taejong)  vector과 When Harry Met Sally... (1989) 확인
taejong, harry = user_to_idx["Taejong"], movie_to_idx["When Harry Met Sally... (1989)"]
taejong_vector, aladdin_vector = als_model.user_factors[taejong], als_model.item_factors[harry]

In [281]:
taejong_vector

array([ 8.42599124e-02,  3.08123618e-01, -4.80960041e-01, -7.70504892e-01,
        4.55295056e-01,  1.65119022e-01,  2.12424397e-01, -2.20252946e-01,
        3.15821469e-01,  5.98911405e-01, -1.52001575e-01,  6.67313397e-01,
        3.99563819e-01,  2.22136557e-01, -4.88440305e-01, -4.68620867e-01,
       -4.31409568e-01,  5.88452041e-01,  4.22213048e-01,  7.52333820e-01,
        5.98816156e-01,  2.29657650e-01,  1.05546981e-01,  9.12428200e-01,
       -5.13866961e-01,  5.14174759e-01,  2.75413275e-01, -5.18804371e-01,
        3.11761349e-01,  4.90929395e-01, -1.46398738e-01, -3.81059766e-01,
       -3.06478478e-02, -2.87878424e-01,  5.27877510e-01, -7.22816467e-01,
       -1.10223722e+00, -4.50531334e-01, -4.14747059e-01,  2.62370557e-01,
        1.50488570e-01,  3.12360018e-01,  5.31804226e-02,  3.57757397e-02,
       -6.42695069e-01,  1.32805482e-01, -3.12333524e-01,  1.04282939e+00,
        2.83589572e-01,  8.77630934e-02,  8.70551243e-02, -9.17694569e-01,
        1.58304125e-01,  

In [282]:
aladdin_vector

array([-9.42420214e-03,  1.46307983e-02, -3.14703648e-04, -4.71641310e-03,
        6.04150770e-03,  7.71319447e-03,  1.56708043e-02,  1.95928123e-02,
        6.30931556e-03,  2.83026267e-02, -2.77050887e-03,  1.23205483e-02,
        7.75552914e-03,  7.04481406e-03, -3.88570916e-04, -2.09724857e-03,
       -1.70499254e-02,  1.44002875e-02,  1.09264273e-02,  1.72002707e-02,
        2.17193421e-02, -1.35513744e-03,  9.43669211e-03,  2.15574428e-02,
       -1.88775845e-02,  2.63369605e-02,  2.02089883e-02, -1.49300732e-02,
        2.28104293e-02,  6.25519315e-04,  9.96695738e-03,  1.06205493e-02,
        3.33633125e-02,  1.27594909e-02,  3.00915129e-02, -1.47077302e-03,
       -2.18284093e-02,  2.20691729e-02, -7.45828403e-03, -4.79087839e-03,
       -7.82622583e-03,  1.56295858e-02,  2.72750463e-02,  6.19850960e-03,
       -9.64259170e-03,  8.02778266e-03, -2.32458599e-02,  3.15009095e-02,
       -4.93109180e-03, -7.83073774e-04,  3.18239536e-03, -1.50187472e-02,
        1.71398968e-02,  

In [283]:
np.dot(taejong_vector,aladdin_vector)

0.6086867

In [284]:
#snow white and the seven dwarfs vector을 가지고 선호도 예측 확인
# 전혀 상관없는 영화 사용해봄
snow = movie_to_idx['Snow White and the Seven Dwarfs (1937)']
snow_vector = als_model.item_factors[snow]

print(np.dot(taejong_vector, snow_vector))

0.09502737


### 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [285]:
idx_to_movie = {v:k for k, v in movie_to_idx.items()}  # movie_id 에서 title을 찾아내는 딕셔너리

def get_similar_movie(movie_name : str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie[1:]  # 내가 넣은 영화는 제외

In [286]:
# 내가 좋아하는 영화와 비슷한 영화
# 내가 넣은 영화는 제외하고 유사한 영화만 출력
get_similar_movie("When Harry Met Sally... (1989)")

["Ferris Bueller's Day Off (1986)",
 'Gate of Heavenly Peace, The (1995)',
 'Free Willy 3: The Rescue (1997)',
 'Alien Escape (1995)',
 'Giant Gila Monster, The (1959)',
 'Cabinet of Dr. Ramirez, The (1991)',
 'Midaq Alley (Callejón de los milagros, El) (1995)',
 'Woo (1998)',
 'Oxygen (1999)']

### 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [287]:
user = user_to_idx["Taejong"]
# recommend에서는 user*item CSR Matrix를 받는다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items = True)
movie_recommended

[(33, 0.3470786),
 (10, 0.30236813),
 (50, 0.2598582),
 (191, 0.25631785),
 (157, 0.21927714),
 (1144, 0.21306053),
 (30, 0.20522968),
 (19, 0.1985277),
 (13, 0.19441968),
 (545, 0.192971),
 (169, 0.18892914),
 (582, 0.1876348),
 (596, 0.17728232),
 (4, 0.1766702),
 (34, 0.16474399),
 (369, 0.1622696),
 (458, 0.15154257),
 (1023, 0.1511209),
 (107, 0.14833921),
 (653, 0.14651933)]

In [288]:
# 내가 추천받은 영화 제목 뽑기
[idx_to_movie[i[0]] for i in movie_recommended] 

['Aladdin (1992)',
 'Beauty and the Beast (1991)',
 'Toy Story 2 (1999)',
 'Little Mermaid, The (1989)',
 'Shawshank Redemption, The (1994)',
 'Bull Durham (1988)',
 'Antz (1998)',
 'Big (1988)',
 "Ferris Bueller's Day Off (1986)",
 'Santa Clause, The (1994)',
 'Fish Called Wanda, A (1988)',
 'Beetlejuice (1988)',
 'Hook (1991)',
 "Bug's Life, A (1998)",
 'Mulan (1998)',
 'Witness (1985)',
 'Mask, The (1994)',
 'Office Space (1999)',
 'Jurassic Park (1993)',
 '12 Angry Men (1957)']

In [289]:
# 영화을 넣으면 비슷한 영화 출력하기
aladin = movie_to_idx["Raising Arizona (1987)"]
explain = als_model.explain(user, csr_data, itemid=aladin)
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Lion King, The (1994)', 0.02767795755934821),
 ('Toy Story (1995)', 0.0131895067893962),
 ('Amadeus (1984)', 0.010347548754542472),
 ('When Harry Met Sally... (1989)', 0.008455536526985514),
 ('Jumanji (1995)', -0.008478559045393422)]

In [290]:
explain[0]

0.05119199058487897

## 회고

-------------------------

>크게 어려운 부분이 없었고 추천도 어느정도 내가 생각한 대로 나오기도 했다. 하지만 crs matrix로 변환하는 과정이 순탄치가 않았다. 처음에는 row data를 merge하지 않고 진행하해서 rating  data에 이미 존재하는 movie_id를 그대로 활용하려고 했지만 정말 이상하게도 csr행렬 변환이 되지 않았다. 이것저것 만져보고 다른 분들과 인자들을 비교도 해보았지만 입력한 파라미터들의 크기는 동일했다. 유일하게 다른것은 title을 직접 변환시키지 않고 movie_id를 그대로 활용했다는 점이었는데 사실 이부분이 왜 오류가 나는건지 아직도 이해가 가지 않는다. 결국 나도 ratings 데이터와 movies 데이터를 merge하여 title을 직접 index화 하였다. 그렇게 하니까 신기하게도 문제가 해결되었다. 

